#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter cTV Inventory Availability Dashboard Parameters
The cTV Audience Affinity dashboard is designed to give clients insights into which cTV apps their audiences have a high affinity for using.  The goal of this dashboard is to provide some assistance with the lack of audience targeting for cTV within DV360.
 1. Find instructions and recommendations for this dashboard <a href="https://docs.google.com/document/d/120kcR9OrS4hGdTxRK0Ig2koNmm6Gl7sH0L6U56N0SAM/view?usp=sharing" target="_blank">here</a>
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'dataset': '',  # Place where tables will be written in BigQuery.
  'recipe_project': '',  # Project where BigQuery dataset will be created.
  'auth_read': 'user',  # Credentials used for reading data.
  'auth_write': 'service',  # Credentials used for writing data.
  'partner_id': '',  # DV360 Partner id.
  'recipe_name': '',  # Name of document to deploy to.
  'audience_ids': '',  # Comma separated list of Audience Ids
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute cTV Inventory Availability Dashboard
This does NOT need to be modified unles you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'drive': {
      'auth': 'user',
      'copy': {
        'destination': {'field': {'description': 'Name of document to deploy to.','order': 1,'name': 'recipe_name','default': '','prefix': 'cTV App Match Table ','kind': 'string'}},
        'source': 'https://docs.google.com/spreadsheets/d/1PPPk2b4gGJHNgQ4hXLiTKzH8pRIdlF5fNy9VCw1v7tM/'
      }
    }
  },
  {
    'dataset': {
      'auth': 'user',
      'dataset': {'field': {'description': 'BigQuery Dataset where all data will live.','name': 'dataset','default': '','order': 3,'kind': 'string'}}
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'body': {
          'params': {
            'type': 'TYPE_INVENTORY_AVAILABILITY',
            'groupBys': [
              'FILTER_APP_URL'
            ],
            'metrics': [
              'METRIC_BID_REQUESTS',
              'METRIC_UNIQUE_VISITORS_COOKIES'
            ],
            'filters': [
              {
                'type': 'FILTER_PARTNER',
                'value': {'field': {'description': 'DV360 Partner id.','name': 'partner_id','order': 1,'kind': 'integer'}}
              },
              {
                'type': 'FILTER_INVENTORY_FORMAT',
                'value': 'VIDEO'
              },
              {
                'type': 'FILTER_COUNTRY',
                'value': 'US'
              }
            ],
            'includeInviteData': True
          },
          'kind': 'doubleclickbidmanager#query',
          'timezoneCode': 'America/Los_Angeles',
          'metadata': {
            'dataRange': 'LAST_30_DAYS',
            'title': {'field': {'name': 'recipe_name','prefix': 'us_country_app_','kind': 'string'}},
            'sendNotification': False,
            'format': 'CSV'
          },
          'schedule': {
            'frequency': 'DAILY',
            'nextRunTimezoneCode': 'America/Los_Angeles',
            'endTimeMs': 7983727200000,
            'nextRunMinuteOfDay': 0
          }
        }
      },
      'out': {
        'bigquery': {
          'table': 'us_country_app',
          'dataset': {'field': {'description': 'BigQuery Dataset where all data will live.','name': 'dataset','default': '','order': 3,'kind': 'string'}},
          'schema': [
            {
              'type': 'STRING',
              'name': 'app_url',
              'mode': 'NULLABLE'
            },
            {
              'type': 'STRING',
              'name': 'impressions',
              'mode': 'NULLABLE'
            },
            {
              'type': 'STRING',
              'name': 'uniques',
              'mode': 'NULLABLE'
            }
          ]
        }
      }
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'body': {
          'params': {
            'type': 'TYPE_INVENTORY_AVAILABILITY',
            'filters': [
              {
                'type': 'FILTER_PARTNER',
                'value': {'field': {'description': 'DV360 Partner id.','name': 'partner_id','order': 1,'kind': 'integer'}}
              },
              {
                'type': 'FILTER_COUNTRY',
                'value': 'US'
              }
            ],
            'metrics': [
              'METRIC_BID_REQUESTS',
              'METRIC_UNIQUE_VISITORS_COOKIES'
            ],
            'includeInviteData': True
          },
          'kind': 'doubleclickbidmanager#query',
          'timezoneCode': 'America/Los_Angeles',
          'metadata': {
            'dataRange': 'LAST_30_DAYS',
            'title': {'field': {'name': 'recipe_name','prefix': 'us_country_baseline_','kind': 'string'}},
            'sendNotification': False,
            'format': 'CSV'
          },
          'schedule': {
            'frequency': 'DAILY',
            'nextRunTimezoneCode': 'America/Los_Angeles',
            'endTimeMs': 7983727200000,
            'nextRunMinuteOfDay': 0
          }
        }
      },
      'out': {
        'bigquery': {
          'table': 'us_country_baseline',
          'dataset': {'field': {'description': 'BigQuery Dataset where all data will live.','name': 'dataset','default': '','order': 3,'kind': 'string'}},
          'schema': [
            {
              'type': 'STRING',
              'name': 'impressions',
              'mode': 'NULLABLE'
            },
            {
              'type': 'STRING',
              'name': 'uniques',
              'mode': 'NULLABLE'
            }
          ]
        }
      }
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'filters': {
          'FILTER_USER_LIST': {
            'values': {'field': {'description': 'Comma separated list of Audience Ids','name': 'audience_ids','order': 2,'kind': 'integer_list'}},
            'single_cell': True
          }
        },
        'body': {
          'params': {
            'type': 'TYPE_INVENTORY_AVAILABILITY',
            'groupBys': [
              'FILTER_AUDIENCE_LIST'
            ],
            'metrics': [
              'METRIC_BID_REQUESTS',
              'METRIC_UNIQUE_VISITORS_COOKIES'
            ],
            'filters': [
              {
                'type': 'FILTER_PARTNER',
                'value': {'field': {'description': 'DV360 Partner id.','name': 'partner_id','order': 1,'kind': 'integer'}}
              },
              {
                'type': 'FILTER_COUNTRY',
                'value': 'US'
              }
            ],
            'includeInviteData': True
          },
          'kind': 'doubleclickbidmanager#query',
          'timezoneCode': 'America/Los_Angeles',
          'metadata': {
            'dataRange': 'LAST_30_DAYS',
            'title': {'field': {'name': 'recipe_name','prefix': 'us_audience_baseline_','kind': 'string'}},
            'sendNotification': False,
            'format': 'CSV'
          },
          'schedule': {
            'frequency': 'DAILY',
            'nextRunTimezoneCode': 'America/Los_Angeles',
            'endTimeMs': 7983727200000,
            'nextRunMinuteOfDay': 0
          }
        }
      },
      'out': {
        'bigquery': {
          'table': 'us_audience_baseline',
          'dataset': {'field': {'description': 'BigQuery Dataset where all data will live.','name': 'dataset','default': '','order': 3,'kind': 'string'}},
          'schema': [
            {
              'type': 'STRING',
              'name': 'user_list',
              'mode': 'NULLABLE'
            },
            {
              'type': 'STRING',
              'name': 'impressions',
              'mode': 'NULLABLE'
            },
            {
              'type': 'STRING',
              'name': 'uniques',
              'mode': 'NULLABLE'
            }
          ]
        }
      }
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'filters': {
          'FILTER_USER_LIST': {
            'values': {'field': {'description': 'Comma separated list of Audience Ids','name': 'audience_ids','order': 2,'kind': 'integer_list'}},
            'single_cell': True
          }
        },
        'body': {
          'params': {
            'type': 'TYPE_INVENTORY_AVAILABILITY',
            'groupBys': [
              'FILTER_APP_URL',
              'FILTER_AUDIENCE_LIST'
            ],
            'metrics': [
              'METRIC_BID_REQUESTS',
              'METRIC_UNIQUE_VISITORS_COOKIES'
            ],
            'filters': [
              {
                'type': 'FILTER_PARTNER',
                'value': {'field': {'description': 'DV360 Partner id.','name': 'partner_id','order': 1,'kind': 'integer'}}
              },
              {
                'type': 'FILTER_INVENTORY_FORMAT',
                'value': 'VIDEO'
              },
              {
                'type': 'FILTER_COUNTRY',
                'value': 'US'
              }
            ],
            'includeInviteData': True
          },
          'kind': 'doubleclickbidmanager#query',
          'timezoneCode': 'America/Los_Angeles',
          'metadata': {
            'dataRange': 'LAST_30_DAYS',
            'title': {'field': {'name': 'recipe_name','prefix': 'us_audience_app_','kind': 'string'}},
            'sendNotification': False,
            'format': 'CSV'
          },
          'schedule': {
            'frequency': 'DAILY',
            'nextRunTimezoneCode': 'America/Los_Angeles',
            'endTimeMs': 7983727200000,
            'nextRunMinuteOfDay': 0
          }
        }
      },
      'out': {
        'bigquery': {
          'table': 'us_audience_app',
          'dataset': {'field': {'description': 'BigQuery Dataset where all data will live.','name': 'dataset','default': '','order': 3,'kind': 'string'}},
          'schema': [
            {
              'type': 'STRING',
              'name': 'app_url',
              'mode': 'NULLABLE'
            },
            {
              'type': 'STRING',
              'name': 'user_list',
              'mode': 'NULLABLE'
            },
            {
              'type': 'STRING',
              'name': 'impressions',
              'mode': 'NULLABLE'
            },
            {
              'type': 'STRING',
              'name': 'uniques',
              'mode': 'NULLABLE'
            }
          ]
        }
      }
    }
  },
  {
    'sheets': {
      'tab': 'data',
      'sheet': {'field': {'description': 'Name of document to deploy to.','order': 1,'name': 'recipe_name','default': '','prefix': 'cTV App Match Table ','kind': 'string'}},
      'header': True,
      'auth': 'user',
      'out': {
        'auth': 'user',
        'bigquery': {
          'table': 'CTV_App_Lookup',
          'dataset': {'field': {'description': 'BigQuery Dataset where all data will live.','name': 'dataset','kind': 'string'}},
          'schema': [
            {
              'type': 'STRING',
              'name': 'Publisher_Name',
              'mode': 'NULLABLE'
            },
            {
              'type': 'STRING',
              'name': 'CTV_App_name',
              'mode': 'NULLABLE'
            }
          ]
        }
      },
      'range': 'A:Z'
    }
  },
  {
    'bigquery': {
      'auth': 'user',
      'description': 'The query to join all the IAR reports into an Affinity Index.',
      'to': {
        'table': 'final_table',
        'dataset': {'field': {'description': 'BigQuery Dataset where all data will live.','name': 'dataset','kind': 'string'}}
      },
      'from': {
        'query': "SELECT    audience_app.app_url,    audience_app.ctv_app_name,  IF    (audience_app.app_url LIKE '%Android%'      OR audience_app.app_url LIKE '%iOS',      'App',      'Domain') AS app_or_domain,    audience_app.user_list AS audience_list,    audience_app.Potential_Impressions AS audience_app_impressions,    audience_app.Unique_Cookies_With_Impressions AS audience_app_uniques,    audience_baseline.Potential_Impressions AS audience_baseline_impressions,    audience_baseline.Unique_Cookies_With_Impressions AS audience_baseline_uniques,    country_app.Potential_Impressions AS country_app_impressions,    country_app.Unique_Cookies_With_Impressions AS country_app_uniques,    country_baseline.Potential_Impressions AS country_baseline_impressions,    country_baseline.Unique_Cookies_With_Impressions AS country_baseline_uniques,    ((audience_app.Unique_Cookies_With_Impressions/NULLIF(audience_baseline.Unique_Cookies_With_Impressions,          0))/NULLIF((country_app.Unique_Cookies_With_Impressions/NULLIF(CAST(country_baseline.Unique_Cookies_With_Impressions AS int64),            0)),        0))*100 AS affinity_index  FROM (    SELECT      user_list,      CAST(      IF        (impressions LIKE '%< 1000%',          0,          CAST(impressions AS int64)) AS int64) AS potential_impressions,      CAST(      IF        (uniques LIKE '%< 100%',          0,          CAST(uniques AS int64)) AS int64) AS unique_cookies_with_impressions    FROM      `[PARAMETER].[PARAMETER].us_audience_baseline` ) AS audience_baseline  JOIN (    SELECT      ctv_app.CTV_App_name AS ctv_app_name,      user_list,      app_url,      CAST(      IF        (impressions LIKE '%< 1000%',          0,          CAST(impressions AS int64)) AS int64) AS potential_impressions,      CAST(      IF        (uniques LIKE '%< 1000%',          0,          CAST(uniques AS int64)) AS int64) AS unique_cookies_with_impressions    FROM      `[PARAMETER].[PARAMETER].us_audience_app` AS a    LEFT JOIN      `[PARAMETER].[PARAMETER].CTV_App_Lookup` AS ctv_app    ON      a.app_url = ctv_app.Publisher_Name ) AS audience_app  ON    audience_baseline.user_list = audience_app.user_list  LEFT JOIN (    SELECT      app_url,      CAST(      IF        (CAST(impressions AS STRING) LIKE '%< 1000%',          0,          CAST(impressions AS int64)) AS int64) AS Potential_Impressions,      CAST(      IF        (CAST(uniques AS STRING) LIKE '%< 1000%',          0,          CAST(uniques AS int64)) AS int64) AS Unique_Cookies_With_Impressions    FROM      `[PARAMETER].[PARAMETER].us_country_app` ) AS country_app  ON    country_app.app_url = audience_app.app_url  CROSS JOIN (    SELECT      CAST(      IF        (CAST(impressions AS STRING) LIKE '%< 1000%',          0,          CAST(impressions AS int64)) AS int64) AS Potential_Impressions,      CAST(      IF        (CAST(uniques AS STRING) LIKE '%< 1000%',          0,          CAST(uniques AS int64)) AS int64) AS Unique_Cookies_With_Impressions    FROM      `[PARAMETER].[PARAMETER].us_country_baseline` ) AS country_baseline",
        'legacy': False,
        'parameters': [
          {'field': {'description': 'Project where BigQuery dataset will be created.','name': 'recipe_project','kind': 'string'}},
          {'field': {'description': 'Place where tables will be written in BigQuery.','name': 'dataset','kind': 'string'}},
          {'field': {'description': 'Project where BigQuery dataset will be created.','name': 'recipe_project','kind': 'string'}},
          {'field': {'description': 'Place where tables will be written in BigQuery.','name': 'dataset','kind': 'string'}},
          {'field': {'description': 'Project where BigQuery dataset will be created.','name': 'recipe_project','kind': 'string'}},
          {'field': {'description': 'Place where tables will be written in BigQuery.','name': 'dataset','kind': 'string'}},
          {'field': {'description': 'Project where BigQuery dataset will be created.','name': 'recipe_project','kind': 'string'}},
          {'field': {'description': 'Place where tables will be written in BigQuery.','name': 'dataset','kind': 'string'}},
          {'field': {'description': 'Project where BigQuery dataset will be created.','name': 'recipe_project','kind': 'string'}},
          {'field': {'description': 'Place where tables will be written in BigQuery.','name': 'dataset','kind': 'string'}},
          {'field': {'description': 'Project where BigQuery dataset will be created.','name': 'recipe_project','kind': 'string'}},
          {'field': {'description': 'Place where tables will be written in BigQuery.','name': 'dataset','kind': 'string'}}
        ]
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True, _force=True)
project.execute(_force=True)
